In [1]:

from pydantic import BaseModel, field_validator
from pydantic import StrictStr, StrictBool, StrictFloat, conlist
from typing import Union, Optional
import json



Create some sample data, 
- one with the field working, 
- one with it broken. 

In [2]:
sample_data = {
    "@context":'_context_',
    "id":'my-test',
    "type":['wcrp:test'],
    "label": "My Test",
    "description": "A sample JSONLD file to test the contents of. ",
    "num":32,
    "nominal_resolution":'1000km'
}


sample_data_broken = {
    "@context":'_context_',
    "id":'my_tes t',
    "type":33,
    "label": "My Test is super long, and above a certain char limit. ",
    "description": "A sample JSONLD file to test the contents of. ",
    "num":31,
    "nominal_resolution":'1e3km'
}


def test_broken(key):
    dummy = {**sample_data}
    dummy[key] = sample_data_broken[key]
    return dummy

In [3]:


# https://www.getorchestra.io/guides/pydantic-validators-custom-validation-functions-using-validator-decorator-in-fastapi

import re
idstr = re.compile(r'[A-z\-]+')
max_id = 30


class cmipld_id:
    @field_validator('id', mode='before')  
    @classmethod
    def id_hypehnate(cls,value):
        assert '_' not in value
        return value
        
    @field_validator('id', mode='after')  
    @classmethod
    def id_allowed_char(cls,value):
        if idstr.fullmatch(value) == None:
            raise UnicodeError(f"Only use A-z and '-' only. [\"id\": \"{value}\"] has the invalid character: ({set(idstr.sub('',value))}).")
        return value
    
    @field_validator('id', mode='after')  
    @classmethod
    def id_max_len(cls,value):
        assert len(value)<= max_id
        return value   
        

class universal_nominal_resolution:
    '''
    A class to check the nominal resolution from the wcrp universal files. 
    You need to know where the repository for these options is stored (almost certainly the wcrp-universe universal items repo)
    '''
    @field_validator('nominal_resolution', mode='after')  
    @classmethod
    def nominal_resolution_check(cls,value):
        from cmipld import processor
        import requests
        # cmipld.processor.get(f'universal:resolution/{value}',depth=1)
        try:
            if value[:5] !='https':
                # if the id is not a direct https link, get the relevant extention usining the ones registered in the cmipld lib. 
                url = processor.resolve_prefix(f'universal:resolution/{value}')
                print('add silent option for processor resolve prefix')
            else: 
                url = value
            # get the file if it exists. This does not check the accuracy of the jsonld file
            return requests.get(url).json()
        # not this return value substitutes the returned information into our pydantic object. 
        
        
        except Exception as e:
            raise FileExistsError(f'Failed to load file of id="{value}" from {url}.json')
        return value
        
        
        

In [9]:


import cmipld

owner = 'ror-community'
repo = 'ror-records'

tag = cmipld.utils.git.get_tags(owner,repo)[0]['name']

rors = [i['name'].replace('.json','') for i in cmipld.utils.git.get_contents(owner,repo,tag)]

In [10]:
len(rors)

1000

In [ ]:
len

In [4]:


# Type checking happens here. Remember to add any extending classes in the brackets after "BaseModel"
class EMD(BaseModel,cmipld_id,universal_nominal_resolution):
    ''' 
    A class containing all the relevant tests for the EMD 
    
    - Ensure all additional test clases are added to the brackets above
    - Ensure each key (optional or not) is added below, and typed correctly. 
    
    '''
    ###################################
    # The entry typing is included here
    ###################################  
    
    # required keys  
    id: StrictStr
    nominal_resolution: StrictStr 
    
    # optional keys
    # _@context: Optional[StrictStr] = None
    # type can be either a str or a list. 
    type: Optional[Union[StrictStr, list]] = ''
    num: Optional[int] = ''
    
    

    def json(self):
        return self.model_json_dump()
    
# Example usage
# a = EMD(**{'num':'1','id':'ts-t'})

In [5]:
for key in sample_data.keys():
    print('#'*50)
    print(key)
    print('-'*50)
    try:
        EMD(**test_broken(key))
    except Exception as e:
        print(e)
    print('#'*50)
    print('')

##################################################
@context
--------------------------------------------------


/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Substituting prefix:
universal: https://wcrp-cmip.github.io/WCRP-universe/resolution/1000km
add silent option for processor resolve prefix
##################################################

##################################################
id
--------------------------------------------------
Substituting prefix:
universal: https://wcrp-cmip.github.io/WCRP-universe/resolution/1000km
add silent option for processor resolve prefix
1 validation error for EMD
id
  Assertion failed,  [type=assertion_error, input_value='my_tes t', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/assertion_error
##################################################

##################################################
type
--------------------------------------------------
Substituting prefix:
universal: https://wcrp-cmip.github.io/WCRP-universe/resolution/1000km
add silent option for processor resolve prefix
2 validation errors for EMD
type.str
  Input should be a valid string

In [6]:
a = EMD(**sample_data)
dict(a)

Substituting prefix:
universal: https://wcrp-cmip.github.io/WCRP-universe/resolution/1000km
add silent option for processor resolve prefix


{'id': 'my-test',
 'nominal_resolution': {'@context': '_context_',
  'description': 'Resolution of 1000 km',
  'id': '1000km',
  'type': ['wcrp:resolution', 'universal'],
  'unit': 'km',
  'value': '1000',
  'label': '1000 km'},
 'type': ['wcrp:test'],
 'num': 32}

In [7]:
a.model

AttributeError: 'EMD' object has no attribute 'model'

In [ ]:
# from pydantic import BaseModel, validator, root_validator, Field
# from typing import Optional

# class ProductItem(BaseModel):
#     name: str
#     price: float
#     stock_quantity: int
#     category: str

#     # Validator for price: it should be a positive number
#     @validator('price')
#     def validate_price(cls, v):
#         if v <= 0:
#             raise ValueError('Price must be positive')
#         return v

#     # Validator for stock_quantity: it should be a non-negative integer
#     @validator('stock_quantity')
#     def validate_stock_quantity(cls, v):
#         if v < 0:
#             raise ValueError('Stock quantity cannot be negative')
#         return v

    
#     # Root validator for name: it should not be empty
#     @root_validator
#     def check_name_not_empty(cls, values):
#         name = values.get('name')
#         if not name or name.strip() == '':
#             raise ValueError('Name cannot be empty')
#         return values

# # Use case: Testing the validation

# def test_product_validation(product_data: dict):
#     try:
#         # Attempt to create a ProductItem instance with the provided dictionary
#         product = ProductItem(**product_data)
#         print("Product is valid:", product)
#     except ValueError as e:
#         print("Validation error:", e)

# # Example usage:


# # Run the tests
# test_product_validation(valid_product)        # Should pass
# test_product_validation(invalid_product_1)    # Should fail due to negative price
# test_product_validation(invalid_product_2)    # Should fail due to invalid category
# test_product_validation(invalid_product_3)    # Should fail due to empty name


In [4]:

# # Test Case 4: Invalid Product with Empty Name
# def test_invalid_product_empty_name():
#     invalid_product = {
#         "name": "",
#         "price": 19.99,
#         "stock_quantity": 50,
#         "category": "clothing"
#     }

#     with pytest.raises(ValidationError) as exc_info:
#         ProductItem(**invalid_product)

#     assert "Name cannot be empty" in str(exc_info.value)


# # Programmatically run specific tests
# if __name__ == "__main__":
#     # Run specific test functions, e.g., 'test_invalid_product_negative_price' and 'test_valid_product'
#     exit_code = pytest.main(["-v", "test_invalid_product_negative_price", "test_valid_product"])
    
#     # Exit with the pytest result code
#     exit(exit_code)

In [175]:

# Layout of a field test class. 

# class to check the num field
class emd_num:
    
    # tell the test to use the test field
    @field_validator('num', mode='after')  
    @classmethod
    # name of function
    def is_even(cls, value: int) -> int:
        '''
        A test to check if number is even
        '''
        # some check 
        if value % 2:
            raise ValueError(f'{value} is not an even number')
        
        # must return something for the test to pass 
        return value  
    
    # repeat for other tests
    @field_validator('num', mode='after')  
    @classmethod
    def is_4(cls, value: int) -> int:
        ''' 
        A test to check if number field == 4 
        '''
        if value != 4 :
            raise ValueError(f'{value} is not 4')
        
        return value  

